In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.optimize import curve_fit

from typing import Literal, Union

from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import matplotlib as mpl

from qs_mps.utils import create_sequential_colors, load_list_of_lists, anim, get_cx, get_cy
from qs_mps.mps_class import MPS

# default parameters of the plot layout
plt.rcParams["text.usetex"] = True  # use latex
plt.rcParams["font.size"] = 13
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.constrained_layout.use"] = True

font = {'family': 'serif', 'size': 20}
plt.rcParams.update({'font.family': font['family'], 'font.size': font['size']})

model = "Z2_dual"
path = "marcos"
# path = "pc"
cx = [3,37]
cy = [3,3]
# cx = None
# cy = None

if model == "Ising":
    model_path = "0_ISING"

elif model == "Z2_dual":
    model_path = "1_Z2"
    param_label = "h"
    boundcond = "obc"
    if cx == None:
        sector = "vacuum_sector"
    else:
        sector = f"{len(cx)}_particle(s)_sector"

elif model == "ANNNI":
    model_path = "2_ANNNI"

elif model == "Cluster":
    model_path = "3_CLUSTER"
else:
    raise SyntaxError("Model not valid. Choose among 'Ising', 'Z2', 'ANNNI', 'Cluster'")
# take the path and precision to save files
# if we want to save the tensors we save them locally because they occupy a lot of memory
if path == "pc":
    path_tensor = f"C:/Users/HP/Desktop/projects/1_Z2"
    parent_path = path_tensor
    path_figures = "G:/My Drive/projects/1_Z2"

    # parent_path = "G:/My Drive/projects/1_Z2"
    # path_tensor = "D:/code/projects/1_Z2"
elif path == "mac":
    # parent_path = "/Users/fradm98/Google Drive/My Drive/projects/1_Z2"
    path_tensor = "/Users/fradm98/Desktop/projects/1_Z2"
    parent_path = path_tensor
elif path == "marcos":
    path_figures = "/Users/fradm/Google Drive/My Drive/projects/1_Z2"
    path_tensor = "/Users/fradm/Desktop/projects/1_Z2"
    parent_path = path_tensor

## Quantum Mutual Information

Let us get a quantum state in mps form and get the reduced density matrix relative to a particular column.
Compute the entropy with it and then trace out the parts of the column to isolate and compute again the entropies of the relative parts. Sum the two entropies of the subsystem and subtract with the initial entropy of the column.

In [12]:
L = 5
l = 3
chi = 16
h = 2
bc = "pbc"

precision = 2
mps = MPS(L=L, d=2**l, model="Z2_dual", chi=chi, h=h, bc=bc)
sector_vac = "vacuum_sector"
cx_vac = np.nan
cy_vac = np.nan
if sector_vac != "vacuum_sector":
    mps.Z2.add_charges(cx_vac, cy_vac)
    mps.charges = mps.Z2.charges
    mps.Z2._define_sector()
else:
    mps.Z2._define_sector()

try:
    mps.load_sites(
        path=path_tensor, precision=precision, cx=cx_vac, cy=cy_vac
    )
    print("State found!!")
except:
    print("State not found! Computing DMRG")
    mps._random_state(seed=3, type_shape="rectangular", chi=chi)
    mps.canonical_form()
    mps.sites.append(np.random.rand(1,2,1))
    mps.L = len(mps.sites)
    energy, entropy, schmidt_vals, t_dmrg = mps.DMRG(trunc_chi=True, trunc_tol=False, where=L//2, long="Z", trans="X")
    mps.check_canonical(site=1)
    aux_qub = mps.sites.pop()
    mps.L -= 1

    mps.order_param()
    mag = mps.mpo_first_moment()
    print(f"initial magentization is: {mag}")

    # mps.sites = [tensor.astype(np.complex128) for tensor in mps.sites]
    mps.save_sites(path=path_tensor, precision=precision, cx=cx_vac, cy=cy_vac)


State not found! Computing DMRG
Sweep n: 0

Sweep n: 1

the tensor at site 6 is in the correct RFC
the tensor at site 5 is in the correct RFC
the tensor at site 4 is in the correct RFC
the tensor at site 3 is in the correct RFC
the tensor at site 2 is in the correct RFC
initial magentization is: (14.970639980346396+0j)
time for saving: 0:00:00.005147


In [13]:
rdm_mid = mps.reduced_density_matrix(sites=[2])
rdm_mid.shape

(8, 8)

In [14]:
from scipy.linalg import logm
(rdm_mid @ logm(rdm_mid)).trace()

(-1.0426035521385251e-05+1.3014139365667713e-18j)

Now we need to trace out the spins we want in the column and find their entropy